In [ ]:
%pip install tensorflow matplotlib scikit-learn pydot

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:

import tensorflow as tf
from tensorflow.keras import layers, activations, losses, Model, Input
from tensorflow.nn import leaky_relu
import numpy as np
from itertools import combinations
from tensorflow.keras.utils import plot_model, Progbar
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

with tf.device('/device:GPU:0'):
  # model architecture
  class RankNet(Model):
      def __init__(self):
          super().__init__()
          self.dense = [layers.Dense(16, activation=leaky_relu), layers.Dense(8, activation=leaky_relu)]
          self.o = layers.Dense(1, activation='linear')
          self.oi_minus_oj = layers.Subtract()
      
      def call(self, inputs):
          xi, xj = inputs
          densei = self.dense[0](xi)
          densej = self.dense[0](xj)
          for dense in self.dense[1:]:
              densei = dense(densei)
              densej = dense(densej)
          oi = self.o(densei)
          oj= self.o(densej)
          oij = self.oi_minus_oj([oi, oj])
          output = layers.Activation('sigmoid')(oij)
          return output
      
      def build_graph(self):
          x = [Input(shape=(10)), Input(shape=(10))]
          return Model(inputs=x, outputs=self.call(x))

# visualize model architecture
plot_model(RankNet().build_graph(), show_shapes=False)

In [ ]:
from os import listdir
from os.path import isfile, join
def getJudgmentsBatchFileByFile():
    """
    Returns a generator function that returns all the judgment batches files from the directory
    """
    files = []
   
    files = [join('./loggedFeatures', file) for file in listdir('./loggedFeatures') if isfile(join('./loggedFeatures', file))]
    for file in files:
        yield file

In [ ]:
import csv
judgments = []
for file in getJudgmentsBatchFileByFile():
  with open(file,'r') as f:
    reader = csv.reader(f, delimiter=' ')
    for row in reader:
      data = []
      for element in row:
        data.append(element.replace(',', ''))
      judgments.append(data)

In [ ]:
judgments.sort(key = lambda judgments: judgments[1])
print(judgments[0])
print(judgments[1])
print(judgments[2])
print(judgments[3])

In [ ]:
for j in judgments:
  del j[2]

print(judgments[0])
print(judgments[1])
print(judgments[2])
print(judgments[3])
print(judgments[4])
print(judgments[5])
print(judgments[6])
print(judgments[7])

In [ ]:
#create pairs
xi = []
xj = []
pij = []
judgmentPairs = []

for i in range(1, len(judgments), 2):
  judgmentPairs.append([judgments[i - 1], judgments[i]])
  xi.append(judgments[i - 1][2:])
  xj.append(judgments[i][2:])
  if judgments[i-1][0] == judgments[i][0]:
    _pij = 0.5
  elif judgments[i-1][0] > judgments[i][0]:
    _pij = 1
  else: 
    _pij = 0
  pij.append(_pij)

In [ ]:
xi = np.array(xi, dtype='float64')

In [ ]:
xj = np.array(xj, dtype='float64')

In [ ]:
pij = np.array(pij, dtype='float64')

In [ ]:
from sklearn.model_selection import train_test_split

xi_train, xi_test, xj_train, xj_test, pij_train, pij_test = train_test_split(
    xi, xj, pij, test_size=0.2, shuffle=False)

In [ ]:
import os

checkpoint_path = "training/ranknet.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)


# train model using compile and fit
with tf.device('/device:GPU:0'):
  ranknet = RankNet()
  ranknet.compile(optimizer='adam', loss='binary_crossentropy')
  history = ranknet.fit([xi_train, xj_train], pij_train, epochs=50, batch_size=1, validation_data=([xi_test, xj_test], pij_test), callbacks=[cp_callback])


In [ ]:
# function for plotting loss
def plot_metrics(train_metric, val_metric=None, metric_name=None, title=None, ylim=5):
    plt.title(title)
    plt.ylim(0,ylim)
    plt.plot(train_metric,color='blue',label=metric_name)
    if val_metric is not None: plt.plot(val_metric,color='green',label='val_' + metric_name)
    plt.legend(loc="upper right")

# plot loss history
plot_metrics(history.history['loss'], history.history['val_loss'], "Loss", "Loss", ylim=0.2)